In [2]:
import pandas as pd
import numpy as np
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import cv2
import glob 
from collections import OrderedDict
import csv

In [ ]:
img = Image.open('图片2.jpg')
data = np.array(img)
luminosityGrey = np.dot(data[...,:3], [0.2989, 0.5870, 0.1140]).astype(np.uint8)

binary = 30 >  luminosityGrey
binaryImg = Image.fromarray(binary.astype(np.uint8) * 255)
binaryImg.save('binary.png')
binaryImg.show()

image = np.ascontiguousarray(binaryImg)
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(image, connectivity=8)
print('num_labels = ',num_labels)
print('======================================')


print('stats = ',stats)
print('======================================')


print('centroids = ',centroids)
print('======================================')


print('labels = ',labels)
print(labels.shape)
print('======================================')


Threshold_a = 60
Noise = []
for i in range(len(stats)):
    if (stats[i][2] + stats[i][3] < Threshold_a):
        Noise.append(i)

print(len(Noise))
print('==========================')


Clean_index = []
for i in range(num_labels):
    temp = i in Noise
    if temp == False:
        Clean_index.append(i)

Clean_index.remove(Clean_index[0])        
print(len(Clean_index))
print(Clean_index)

In [ ]:

for i in range(len(Clean_index)):
    Fracs = np.zeros((image.shape[0], image.shape[1])) != 0 # 全 False
    Frac_index = np.argwhere(labels==Clean_index[i])  # 寻找标签值
    for j in range(len(Frac_index)):
        x = Frac_index[j][0]
        y = Frac_index[j][1]
        Fracs[x][y] = True
    Img_Name = "Fractures/binary" + str(i)+ ".png"
    FracsImg = Image.fromarray(Fracs.astype(np.uint8) * 255)
    FracsImg.save(Img_Name)
    

Fracs = np.zeros((image.shape[0], image.shape[1])) != 0 # 全 False
for i in range(len(Clean_index)):
    Frac_index = np.argwhere(labels==Clean_index[i])
    for j in range(len(Frac_index)):
        x = Frac_index[j][0]
        y = Frac_index[j][1]
        Fracs[x][y] = True
Img_Name = "Fractures/Fracs.png"
FracsImg = Image.fromarray(Fracs.astype(np.uint8) * 255)
FracsImg.save(Img_Name)
FracsImg.show()

In [ ]:

Fuse = [58, 73, 75]
Fracs = np.zeros((image.shape[0], image.shape[1])) != 0 # 全 False
for i in range(len(Fuse)):
    Frac_index = np.argwhere(labels==Fuse[i])
    for j in range(len(Frac_index)):
        x = Frac_index[j][0]
        y = Frac_index[j][1]
        Fracs[x][y] = True
Img_Name = "Fractures/Fracs-fusion.png"
FracsImg = Image.fromarray(Fracs.astype(np.uint8) * 255)
FracsImg.save(Img_Name)
FracsImg.show()

In [ ]:

for i in range(len(Clean_index)):
    Frac_colors = np.zeros((image.shape[0], image.shape[1], 3), np.uint8)
    mask = labels == Clean_index[i]
    Frac_colors[:, :, 0][mask] = np.random.randint(0, 255)
    Frac_colors[:, :, 1][mask] = np.random.randint(0, 255)
    Frac_colors[:, :, 2][mask] = np.random.randint(0, 255)
    
    Img_Name = "Fractures/color" + str(i)+ ".png"
    cv2.imwrite(Img_Name, Frac_colors)


Frac_colors = np.zeros((image.shape[0], image.shape[1], 3), np.uint8)
for x in Clean_index:
    mask = labels == x
    Frac_colors[:, :, 0][mask] = np.random.randint(0, 255)
    Frac_colors[:, :, 1][mask] = np.random.randint(0, 255)
    Frac_colors[:, :, 2][mask] = np.random.randint(0, 255)

cv2.imwrite('Fractures/Frac_colors.png', Frac_colors)
cv2.imshow('Frac_colors', Frac_colors)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
Total_Fracs = np.zeros((image.shape[0], image.shape[1])) != 0
for i in range(len(Clean_index)):
    Frac_index = np.argwhere(labels==Clean_index[i])
    for j in range(len(Frac_index)):
        x = Frac_index[j][0]
        y = Frac_index[j][1]
        Total_Fracs[x][y] = True
Coor = np.argwhere(Total_Fracs==True)
print(Coor.shape)

X = Coor

dbscan = DBSCAN(eps=7, min_samples=3)

Labels = dbscan.fit_predict(X)

plt.scatter(X[:, 0], X[:, 1], c=Labels, s=0.1)
plt.show()

In [ ]:
def split(DF, L):
    G = DF[DF[2] == L]
    G = G.iloc[:, :2]
    G = np.array(G)
    return G
    
def Data_splite_withlabels(Array, Label, Num):
    contact = np.column_stack((Array, Label))
    C = pd.DataFrame(contact)
    group = []
    for i in range(Num):
        G = split(C,i)
        group.append(G)
    group = np.array(group)
    
    return group

Group = Data_splite_withlabels(Coor, Labels, 8)
Arr0 = Group[0]
Arr1 = Group[1]
Arr2 = Group[2]
Arr3 = Group[3]
Arr4 = Group[4]
Arr5 = Group[5]
Arr6 = Group[6]
Arr7 = Group[7]

print(Arr1)

In [ ]:
import cv2
import numpy as np
ConcactImg = Concact_Fractures(Arr0, Arr1)
cv2.imwrite('ConcactImg.png',ConcactImg)
cv2.imshow('Image with Crack Path', ConcactImg)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def Singel_image(Arr):
    image = np.zeros((360, 360), dtype=np.uint8)
    crack_path = Arr
    # 步骤 3: 将通路像素设置为白色
    for x, y in crack_path:
        if 0 <= x < 360 and 0 <= y < 360:
            image[y, x] = 255
    return image

def Concact_Fractures (Arr01, Arr02):
    image = np.zeros((360, 360), dtype=np.uint8)
    crack_path = np.vstack((Arr01,Arr02))
    # 步骤 3: 将通路像素设置为白色
    for x, y in crack_path:
        if 0 <= x < 360 and 0 <= y < 360:
            image[y, x] = 255
    return image

image00 = Singel_image(Arr0)
image01 = Singel_image(Arr1)
image02 = Singel_image(Arr2)
image03 = Singel_image(Arr3)
image04 = Singel_image(Arr4)
image05 = Singel_image(Arr5)
image06 = Singel_image(Arr6)
image07 = Singel_image(Arr7)

In [ ]:
import cv2
import numpy as np
from scipy.optimize import curve_fit

def sin_func(x, A, omega, phi, b):
    return A * np.sin(omega * x + phi) + b

def dfs(image, visited, x, y, component):
    height, width = image.shape
    if x < 0 or x >= width or y < 0 or y >= height or visited[y, x] or image[y, x] == 0:
        return
    visited[y, x] = True
    component.append((x, y))
    directions = [(0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1), (-1, 0), (-1, 1)]
    for dx, dy in directions:
        dfs(image, visited, x + dx, y + dy, component)

def find_all_subcracks(image):
    height, width = image.shape
    visited = np.zeros((height, width), dtype=bool)
    all_subcracks = []
    for y in range(height):
        for x in range(width):
            if image[y, x] == 255 and not visited[y, x]:
                component = []
                dfs(image, visited, x, y, component)
                all_subcracks.append(np.array(component))
    return all_subcracks

def fit_subcrack(subcrack):
    x = subcrack[:, 0]
    y = subcrack[:, 1]
    try:
        initial_guess = [10, 0.1, 0, 180]
        popt, _ = curve_fit(sin_func, x, y, p0=initial_guess)
        return popt
    except:
        return None
def group_subcracks(all_subcracks):
    groups = []
    param_threshold = 0.1
    position_threshold = 20  
    for subcrack in all_subcracks:
        params = fit_subcrack(subcrack)
        if params is not None:
            found_group = False
            for group in groups:
                ref_params = group[0][0]
                ref_subcrack = group[0][1]
                A1, omega1, phi1, _ = params
                A2, omega2, phi2, _ = ref_params

                if (np.abs(A1 - A2) < param_threshold and
                        np.abs(omega1 - omega2) < param_threshold and
                        np.abs(phi1 - phi2) < param_threshold):
                    
                    subcrack_center = np.mean(subcrack, axis=0)
                    ref_subcrack_center = np.mean(ref_subcrack, axis=0)
                    distance = np.linalg.norm(subcrack_center - ref_subcrack_center)
                    if distance < position_threshold:
                        group.append((params, subcrack))
                        found_group = True
                        break
            if not found_group:
                groups.append([(params, subcrack)])
    return groups

def count_original_cracks(image):
    all_subcracks = find_all_subcracks(image)
    groups = group_subcracks(all_subcracks)
    return len(groups)

image = cv2.imread('ConcactImg.png', 0)
_, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

num_original_cracks = count_original_cracks(binary_image)
print(f"原始自然裂缝的数量为: {num_original_cracks}")

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import DBSCAN
import networkx as nx
import torch
from torch_geometric.data import Data
import matplotlib.pyplot as plt

image = np.zeros((360, 360), dtype=np.uint8)

x = np.arange(0, 360)
y = (180 + 50 * np.sin(x * 0.05)).astype(int)
image[y, x] = 255

contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

node_features = []
for contour in contours:
    length = cv2.arcLength(contour, False)
    x, y, w, h = cv2.boundingRect(contour)
    width = w
    M = cv2.moments(contour)
    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
    else:
        cX, cY = 0, 0
    node_features.append([length, width, cX, cY])

num_nodes = len(node_features)
edge_index = []
for i in range(num_nodes):
    for j in range(num_nodes):
        if i != j:
            edge_index.append([i, j])
edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
x = torch.tensor(node_features, dtype=torch.float)
data = Data(x=x, edge_index=edge_index)
X = data.x.numpy()
clustering = DBSCAN(eps=30, min_samples=1).fit(X)
labels = clustering.labels_

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(image, cmap='gray')
plt.title('Original Image')

plt.subplot(1, 2, 2)
colored_image = np.zeros((360, 360, 3), dtype=np.uint8)
for i, contour in enumerate(contours):
    color = [int(c) for c in np.random.randint(0, 256, 3)]
    label = labels[i]
    if label != -1:
        cv2.drawContours(colored_image, [contour], -1, color, 2)
plt.imshow(colored_image)
plt.title('Grouped Cracks')
plt.show()